## Initial Setup

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os
import math
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

import warnings
warnings.filterwarnings("ignore")

## Training Parameters

In [2]:
# Hyperparams if GPU is available
if tf.test.is_gpu_available():
    # GPU
    BATCH_SIZE = 256  # Number of examples used in each iteration
    EPOCHS = 10  # Number of passes through entire dataset
    MAX_LEN = 170  # Max length of review (in words)
    VOCAB_SIZE = 1000  # Size of vocabulary dictionary
    EMBEDDING = 8  # Dimension of word embedding vector
    
# Hyperparams for CPU training
else:
    # CPU
    BATCH_SIZE = 128
    EPOCHS = 10
    MAX_LEN = 170
    VOCAB_SIZE = 1000
    EMBEDDING = 8

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


## Data

In [3]:
data=pd.read_excel(r"C:\Users\User\Desktop\Machine Learning Project\Biodiesel-Yield-Using-ANN-main\Xantham_gum_data.xlsx")
data

,X1,X2,X3,Observed
0,1,2,7.5,7.22
1,3,2,7.5,7.52
2,1,6,7.5,4.58
3,3,6,7.5,6.45
4,1,4,0.0,4.47
5,3,4,0.0,5.67
6,1,4,15.0,6.46
7,3,4,15.0,7.76
8,2,2,0.0,6.65
9,2,6,0.0,4.52


## Data Preprocessing

In [4]:
target="Observed"
X=data.drop(columns=[target])
y=data[target]
X1=X.values
y1=y.values.reshape(-1,1)

In [5]:
def get_preds(y_test, y_preds):
    y_test=pd.DataFrame(y_test)
    y_test.rename(columns={0:'Actual'}, inplace=True)
    y_preds=pd.DataFrame(y_preds)
    y_preds.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([y_test, y_preds], axis=1)
    return predictions

In [6]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score,make_scorer
X_train, X_test, y_train, y_test= train_test_split(X1, y1, test_size=0.2, random_state=66)

In [7]:
sc_x=StandardScaler()
X_train=sc_x.fit_transform(X_train)
X_test=sc_x.transform(X_test)
X1=sc_x.transform(X1)
sc_y=StandardScaler()
y_train=sc_y.fit_transform(y_train)
y_test=sc_y.transform(y_test)
y1=sc_y.transform(y1)

In [8]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [9]:
NN_model = Sequential()

In [10]:
# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(16, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               512       
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [11]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [16]:
import time
start_time=time.time()
NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.0025 - mean_absolute_error: 0.0025
Epoch 1: val_loss did not improve from 0.00482
1/1 [==============================] - 1s 1s/step - loss: 0.0025 - mean_absolute_error: 0.0025 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.1167 - mean_absolute_error: 0.1167
Epoch 2: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 59ms/step - loss: 0.1167 - mean_absolute_error: 0.1167 - val_loss: 0.0484 - val_mean_absolute_error: 0.0484
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.0625 - mean_absolute_error: 0.0625
Epoch 3: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 57ms/step - loss: 0.0625 - mean_absolute_error: 0.0625 - val_loss: 0.1077 - val_mean_absolute_error: 0.1077
Epoch 4/500
1/1 [==============================] - ETA: 0s - loss: 0.0552 - mean_absolute_error: 0.

1/1 [==============================] - ETA: 0s - loss: 0.0098 - mean_absolute_error: 0.0098
Epoch 28: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 91ms/step - loss: 0.0098 - mean_absolute_error: 0.0098 - val_loss: 0.0601 - val_mean_absolute_error: 0.0601
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.0057
Epoch 29: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 90ms/step - loss: 0.0057 - mean_absolute_error: 0.0057 - val_loss: 0.0551 - val_mean_absolute_error: 0.0551
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.0094 - mean_absolute_error: 0.0094
Epoch 30: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 85ms/step - loss: 0.0094 - mean_absolute_error: 0.0094 - val_loss: 0.0582 - val_mean_absolute_error: 0.0582
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.0107 - mean_absolute_error: 0.0107

1/1 [==============================] - ETA: 0s - loss: 0.0202 - mean_absolute_error: 0.0202
Epoch 55: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 70ms/step - loss: 0.0202 - mean_absolute_error: 0.0202 - val_loss: 0.0679 - val_mean_absolute_error: 0.0679
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.0146 - mean_absolute_error: 0.0146
Epoch 56: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.0146 - mean_absolute_error: 0.0146 - val_loss: 0.0490 - val_mean_absolute_error: 0.0490
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.0137 - mean_absolute_error: 0.0137
Epoch 57: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0137 - mean_absolute_error: 0.0137 - val_loss: 0.0444 - val_mean_absolute_error: 0.0444
Epoch 58/500
1/1 [==============================] - ETA: 0s - loss: 0.0201 - mean_absolute_error: 0.0201

1/1 [==============================] - ETA: 0s - loss: 0.0090 - mean_absolute_error: 0.0090
Epoch 82: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.0090 - mean_absolute_error: 0.0090 - val_loss: 0.0604 - val_mean_absolute_error: 0.0604
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.0130 - mean_absolute_error: 0.0130
Epoch 83: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0130 - mean_absolute_error: 0.0130 - val_loss: 0.0629 - val_mean_absolute_error: 0.0629
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.0087 - mean_absolute_error: 0.0087
Epoch 84: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0087 - mean_absolute_error: 0.0087 - val_loss: 0.0716 - val_mean_absolute_error: 0.0716
Epoch 85/500
1/1 [==============================] - ETA: 0s - loss: 0.0124 - mean_absolute_error: 0.0124

1/1 [==============================] - ETA: 0s - loss: 0.0177 - mean_absolute_error: 0.0177
Epoch 109: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.0177 - mean_absolute_error: 0.0177 - val_loss: 0.0725 - val_mean_absolute_error: 0.0725
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.0133 - mean_absolute_error: 0.0133
Epoch 110: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 51ms/step - loss: 0.0133 - mean_absolute_error: 0.0133 - val_loss: 0.0550 - val_mean_absolute_error: 0.0550
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.0132 - mean_absolute_error: 0.0132
Epoch 111: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 56ms/step - loss: 0.0132 - mean_absolute_error: 0.0132 - val_loss: 0.0541 - val_mean_absolute_error: 0.0541
Epoch 112/500
1/1 [==============================] - ETA: 0s - loss: 0.0144 - mean_absolute_error: 

...

Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.0116 - mean_absolute_error: 0.0116
Epoch 379: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.0116 - mean_absolute_error: 0.0116 - val_loss: 0.0549 - val_mean_absolute_error: 0.0549
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.0126 - mean_absolute_error: 0.0126
Epoch 380: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0126 - mean_absolute_error: 0.0126 - val_loss: 0.0671 - val_mean_absolute_error: 0.0671
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.0085 - mean_absolute_error: 0.0085
Epoch 381: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0085 - mean_absolute_error: 0.0085 - val_loss: 0.0693 - val_mean_absolute_error: 0.0693
Epoch 382/500
1/1 [==============================] - ETA: 0s - loss: 0.0103 - mean_ab

Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.0076 - mean_absolute_error: 0.0076
Epoch 406: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 58ms/step - loss: 0.0076 - mean_absolute_error: 0.0076 - val_loss: 0.0607 - val_mean_absolute_error: 0.0607
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.0055
Epoch 407: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 62ms/step - loss: 0.0055 - mean_absolute_error: 0.0055 - val_loss: 0.0661 - val_mean_absolute_error: 0.0661
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0073 - mean_absolute_error: 0.0073
Epoch 408: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 65ms/step - loss: 0.0073 - mean_absolute_error: 0.0073 - val_loss: 0.0636 - val_mean_absolute_error: 0.0636
Epoch 409/500
1/1 [==============================] - ETA: 0s - loss: 0.0065 - mean_ab

Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.0046 - mean_absolute_error: 0.0046
Epoch 433: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 47ms/step - loss: 0.0046 - mean_absolute_error: 0.0046 - val_loss: 0.0637 - val_mean_absolute_error: 0.0637
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.0055 - mean_absolute_error: 0.0055
Epoch 434: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 52ms/step - loss: 0.0055 - mean_absolute_error: 0.0055 - val_loss: 0.0610 - val_mean_absolute_error: 0.0610
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0059 - mean_absolute_error: 0.0059
Epoch 435: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0059 - mean_absolute_error: 0.0059 - val_loss: 0.0560 - val_mean_absolute_error: 0.0560
Epoch 436/500
1/1 [==============================] - ETA: 0s - loss: 0.0061 - mean_ab

Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0095 - mean_absolute_error: 0.0095
Epoch 460: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 79ms/step - loss: 0.0095 - mean_absolute_error: 0.0095 - val_loss: 0.0648 - val_mean_absolute_error: 0.0648
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0083 - mean_absolute_error: 0.0083
Epoch 461: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 66ms/step - loss: 0.0083 - mean_absolute_error: 0.0083 - val_loss: 0.0557 - val_mean_absolute_error: 0.0557
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0083 - mean_absolute_error: 0.0083
Epoch 462: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 55ms/step - loss: 0.0083 - mean_absolute_error: 0.0083 - val_loss: 0.0557 - val_mean_absolute_error: 0.0557
Epoch 463/500
1/1 [==============================] - ETA: 0s - loss: 0.0088 - mean_ab

Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0068 - mean_absolute_error: 0.0068
Epoch 487: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 57ms/step - loss: 0.0068 - mean_absolute_error: 0.0068 - val_loss: 0.0711 - val_mean_absolute_error: 0.0711
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0106 - mean_absolute_error: 0.0106
Epoch 488: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 71ms/step - loss: 0.0106 - mean_absolute_error: 0.0106 - val_loss: 0.0637 - val_mean_absolute_error: 0.0637
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0064 - mean_absolute_error: 0.0064
Epoch 489: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 81ms/step - loss: 0.0064 - mean_absolute_error: 0.0064 - val_loss: 0.0619 - val_mean_absolute_error: 0.0619
Epoch 490/500
1/1 [==============================] - ETA: 0s - loss: 0.0058 - mean_ab

In [17]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [18]:
y_pred=NN_model.predict(X_test)

1/1 [==============================] - 0s 122ms/step


In [19]:
r2_score(y_test,y_pred)

0.8882121997542642

In [20]:
y_test1 = sc_y.inverse_transform(y_test)
pred1 = sc_y.inverse_transform(y_pred.reshape(-1,1))
ann_prediction_test=get_preds(y_test1, pred1)
ann_prediction_test

,Actual,Predicted
0,7.52,7.829707
1,4.47,4.636868
2,5.81,6.281061
3,5.86,6.281061


In [21]:
ann_prediction_test["Actual"].corr(ann_prediction_test["Predicted"])

0.9953379519820674

In [22]:
from math import sqrt
print("TEST Mean absolute error :", mean_absolute_error(y_test1,pred1))
print("TEST Mean squared error :",mean_squared_error(y_test1,pred1))
print("TEST Root Mean squared error :",sqrt(mean_squared_error(y_test1,pred1)))

TEST Mean absolute error : 0.34217437267303485
TEST Mean squared error : 0.13073864618105346
TEST Root Mean squared error : 0.3615779946028982


In [23]:
y_pred_train=NN_model.predict(X_train)

1/1 [==============================] - 0s 35ms/step


In [24]:
r2_score(y_train,y_pred_train)

0.9993320414555981

In [25]:
y_train1 = sc_y.inverse_transform(y_train)
pred_train= sc_y.inverse_transform(y_pred_train.reshape(-1,1))
ann_prediction_train=get_preds(y_train1, pred_train)
ann_prediction_train

,Actual,Predicted
0,6.93,6.926017
1,6.65,6.642723
2,4.58,4.567492
3,4.52,4.511302
4,6.46,6.452944
5,7.76,7.771520
6,7.22,7.221429
7,6.28,6.281061
8,6.45,6.451786
9,5.67,5.662909


In [26]:
ann_prediction_train["Actual"].corr(ann_prediction_train["Predicted"])

0.999702934313484

In [27]:
from math import sqrt
print("Train Mean absolute error :", mean_absolute_error(y_train1,pred_train))
print("Train Mean squared error :",mean_squared_error(y_train1,pred_train))
print("Train Root Mean squared error :",sqrt(mean_squared_error(y_train1,pred_train)))

Train Mean absolute error : 0.01531768652108995
Train Mean squared error : 0.000718413168258935
Train Root Mean squared error : 0.026803230556388814


In [28]:
y_pred_real=NN_model.predict(X1)

1/1 [==============================] - 0s 36ms/step


In [29]:
sc_y.inverse_transform(y_pred_real)

array([[7.221429 ],
       [7.829707 ],
       [4.5674915],
       [6.4517856],
       [4.636868 ],
       [5.6629086],
       [6.4529443],
       [7.7715197],
       [6.642723 ],
       [4.511302 ],
       [8.314599 ],
       [6.9260173],
       [6.281061 ],
       [6.281061 ],
       [6.281061 ],
       [6.281061 ],
       [6.281061 ]], dtype=float32)

In [30]:
r2_score(y1,y_pred_real)

0.9725526854127183

In [31]:
y_real = sc_y.inverse_transform(y1)
pred_real= sc_y.inverse_transform(y_pred_real.reshape(-1,1))
ann_prediction_real=get_preds(y_real, pred_real)
ann_prediction_real

,Actual,Predicted
0,7.22,7.221429
1,7.52,7.829707
2,4.58,4.567492
3,6.45,6.451786
4,4.47,4.636868
5,5.67,5.662909
6,6.46,6.452944
7,7.76,7.771520
8,6.65,6.642723
9,4.52,4.511302


In [32]:
NN_model1= Sequential()

In [33]:
# The Input Layer :
NN_model1.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='leaky_relu'))

# The Hidden Layers :
NN_model1.add(Dense(64, kernel_initializer='normal',activation='leaky_relu'))
NN_model1.add(Dense(32, kernel_initializer='normal',activation='leaky_relu'))
NN_model1.add(Dense(16, kernel_initializer='normal',activation='leaky_relu'))

# The Output Layer :
NN_model1.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               512       
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [34]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint1 = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list1 = [checkpoint1]

In [35]:
import time
start_time=time.time()
NN_model1.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list1)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8342 - mean_absolute_error: 0.8342
Epoch 1: val_loss improved from inf to 0.18429, saving model to Weights-001--0.18429.hdf5
1/1 [==============================] - 2s 2s/step - loss: 0.8342 - mean_absolute_error: 0.8342 - val_loss: 0.1843 - val_mean_absolute_error: 0.1843
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8335 - mean_absolute_error: 0.8335
Epoch 2: val_loss did not improve from 0.18429
1/1 [==============================] - 0s 54ms/step - loss: 0.8335 - mean_absolute_error: 0.8335 - val_loss: 0.1859 - val_mean_absolute_error: 0.1859
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8328 - mean_absolute_error: 0.8328
Epoch 3: val_loss did not improve from 0.18429
1/1 [==============================] - 0s 52ms/step - loss: 0.8328 - mean_absolute_error: 0.8328 - val_loss: 0.1875 - val_mean_absolute_error: 0.1875
Epoch 4/500
1/1 [==============================] - ETA: 

Epoch 28/500
1/1 [==============================] - ETA: 0s - loss: 0.7748 - mean_absolute_error: 0.7748
Epoch 28: val_loss did not improve from 0.18429
1/1 [==============================] - 0s 113ms/step - loss: 0.7748 - mean_absolute_error: 0.7748 - val_loss: 0.2328 - val_mean_absolute_error: 0.2328
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.7679 - mean_absolute_error: 0.7679
Epoch 29: val_loss did not improve from 0.18429
1/1 [==============================] - 0s 116ms/step - loss: 0.7679 - mean_absolute_error: 0.7679 - val_loss: 0.2364 - val_mean_absolute_error: 0.2364
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.7597 - mean_absolute_error: 0.7597
Epoch 30: val_loss did not improve from 0.18429
1/1 [==============================] - 0s 112ms/step - loss: 0.7597 - mean_absolute_error: 0.7597 - val_loss: 0.2409 - val_mean_absolute_error: 0.2409
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.7502 - mean_absolu

1/1 [==============================] - 0s 54ms/step - loss: 0.1954 - mean_absolute_error: 0.1954 - val_loss: 0.0099 - val_mean_absolute_error: 0.0099
Epoch 55/500
1/1 [==============================] - ETA: 0s - loss: 0.1654 - mean_absolute_error: 0.1654
Epoch 55: val_loss improved from 0.00936 to 0.00604, saving model to Weights-055--0.00604.hdf5
1/1 [==============================] - 0s 114ms/step - loss: 0.1654 - mean_absolute_error: 0.1654 - val_loss: 0.0060 - val_mean_absolute_error: 0.0060
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.1652 - mean_absolute_error: 0.1652
Epoch 56: val_loss improved from 0.00604 to 0.00482, saving model to Weights-056--0.00482.hdf5
1/1 [==============================] - 0s 106ms/step - loss: 0.1652 - mean_absolute_error: 0.1652 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.1601 - mean_absolute_error: 0.1601
Epoch 57: val_loss did not improve from 0.0

1/1 [==============================] - ETA: 0s - loss: 0.0711 - mean_absolute_error: 0.0711
Epoch 81: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 118ms/step - loss: 0.0711 - mean_absolute_error: 0.0711 - val_loss: 0.0193 - val_mean_absolute_error: 0.0193
Epoch 82/500
1/1 [==============================] - ETA: 0s - loss: 0.0665 - mean_absolute_error: 0.0665
Epoch 82: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 101ms/step - loss: 0.0665 - mean_absolute_error: 0.0665 - val_loss: 0.0097 - val_mean_absolute_error: 0.0097
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.0612 - mean_absolute_error: 0.0612
Epoch 83: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 110ms/step - loss: 0.0612 - mean_absolute_error: 0.0612 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.0625 - mean_absolute_error: 0.0

Epoch 108/500
1/1 [==============================] - ETA: 0s - loss: 0.0208 - mean_absolute_error: 0.0208
Epoch 108: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 100ms/step - loss: 0.0208 - mean_absolute_error: 0.0208 - val_loss: 0.0634 - val_mean_absolute_error: 0.0634
Epoch 109/500
1/1 [==============================] - ETA: 0s - loss: 0.0215 - mean_absolute_error: 0.0215
Epoch 109: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 96ms/step - loss: 0.0215 - mean_absolute_error: 0.0215 - val_loss: 0.0554 - val_mean_absolute_error: 0.0554
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.0217 - mean_absolute_error: 0.0217
Epoch 110: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 101ms/step - loss: 0.0217 - mean_absolute_error: 0.0217 - val_loss: 0.0633 - val_mean_absolute_error: 0.0633
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.0138 - mean_

...

Epoch 378/500
1/1 [==============================] - ETA: 0s - loss: 0.0089 - mean_absolute_error: 0.0089
Epoch 378: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.0089 - mean_absolute_error: 0.0089 - val_loss: 0.0603 - val_mean_absolute_error: 0.0603
Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.0050 - mean_absolute_error: 0.0050
Epoch 379: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 58ms/step - loss: 0.0050 - mean_absolute_error: 0.0050 - val_loss: 0.0613 - val_mean_absolute_error: 0.0613
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.0071 - mean_absolute_error: 0.0071
Epoch 380: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0071 - mean_absolute_error: 0.0071 - val_loss: 0.0625 - val_mean_absolute_error: 0.0625
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.0087 - mean_ab

Epoch 405/500
1/1 [==============================] - ETA: 0s - loss: 0.0045 - mean_absolute_error: 0.0045
Epoch 405: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.0045 - mean_absolute_error: 0.0045 - val_loss: 0.0694 - val_mean_absolute_error: 0.0694
Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.0087 - mean_absolute_error: 0.0087
Epoch 406: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0087 - mean_absolute_error: 0.0087 - val_loss: 0.0700 - val_mean_absolute_error: 0.0700
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.0121 - mean_absolute_error: 0.0121
Epoch 407: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0121 - mean_absolute_error: 0.0121 - val_loss: 0.0664 - val_mean_absolute_error: 0.0664
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0094 - mean_ab

Epoch 432/500
1/1 [==============================] - ETA: 0s - loss: 0.0123 - mean_absolute_error: 0.0123
Epoch 432: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 86ms/step - loss: 0.0123 - mean_absolute_error: 0.0123 - val_loss: 0.0675 - val_mean_absolute_error: 0.0675
Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.0091
Epoch 433: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 156ms/step - loss: 0.0091 - mean_absolute_error: 0.0091 - val_loss: 0.0591 - val_mean_absolute_error: 0.0591
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.0077 - mean_absolute_error: 0.0077
Epoch 434: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 122ms/step - loss: 0.0077 - mean_absolute_error: 0.0077 - val_loss: 0.0557 - val_mean_absolute_error: 0.0557
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0090 - mean_

Epoch 459/500
1/1 [==============================] - ETA: 0s - loss: 0.0078 - mean_absolute_error: 0.0078
Epoch 459: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.0078 - mean_absolute_error: 0.0078 - val_loss: 0.0674 - val_mean_absolute_error: 0.0674
Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0087 - mean_absolute_error: 0.0087
Epoch 460: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.0087 - mean_absolute_error: 0.0087 - val_loss: 0.0621 - val_mean_absolute_error: 0.0621
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0067 - mean_absolute_error: 0.0067
Epoch 461: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 77ms/step - loss: 0.0067 - mean_absolute_error: 0.0067 - val_loss: 0.0589 - val_mean_absolute_error: 0.0589
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0069 - mean_ab

Epoch 486/500
1/1 [==============================] - ETA: 0s - loss: 0.0069 - mean_absolute_error: 0.0069
Epoch 486: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 71ms/step - loss: 0.0069 - mean_absolute_error: 0.0069 - val_loss: 0.0611 - val_mean_absolute_error: 0.0611
Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0086 - mean_absolute_error: 0.0086
Epoch 487: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 55ms/step - loss: 0.0086 - mean_absolute_error: 0.0086 - val_loss: 0.0608 - val_mean_absolute_error: 0.0608
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0070 - mean_absolute_error: 0.0070
Epoch 488: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 48ms/step - loss: 0.0070 - mean_absolute_error: 0.0070 - val_loss: 0.0620 - val_mean_absolute_error: 0.0620
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0062 - mean_ab

In [36]:
NN_model1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [37]:
y_pred1=NN_model1.predict(X_test)
r2_score(y_test,y_pred1)

1/1 [==============================] - 0s 125ms/step


0.8824588889854778

In [38]:
y_pred_train1=NN_model1.predict(X_train)
r2_score(y_train,y_pred_train1)

1/1 [==============================] - 0s 42ms/step


0.9993884419404706

In [39]:
from math import sqrt
print("Train leaky Mean absolute error :", mean_absolute_error(y_train,y_pred_train1))
print("Train leaky Mean squared error :",mean_squared_error(y_train,y_pred_train1))
print("Train leaky Root Mean squared error :",sqrt(mean_squared_error(y_train,y_pred_train1)))

Train leaky Mean absolute error : 0.016288985394629803
Train leaky Mean squared error : 0.0006115580595293441
Train leaky Root Mean squared error : 0.024729699948227116


In [40]:
y_pred_real1=NN_model1.predict(X1)
r2_score(y1,y_pred_real1)

1/1 [==============================] - 0s 33ms/step


0.9712055195061312

In [41]:
NN_model2= Sequential()

In [42]:
# The Input Layer :
NN_model2.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='exponential'))

# The Hidden Layers :
NN_model2.add(Dense(64, kernel_initializer='normal',activation='exponential'))
NN_model2.add(Dense(32, kernel_initializer='normal',activation='exponential'))
NN_model2.add(Dense(16, kernel_initializer='normal',activation='exponential'))

# The Output Layer :
NN_model2.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               512       
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 16)                528       
                                                                 
 dense_14 (Dense)            (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [43]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint2 = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list2 = [checkpoint2]

In [44]:
import time
start_time=time.time()
NN_model2.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list2)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8792 - mean_absolute_error: 0.8792
Epoch 1: val_loss improved from inf to 0.13649, saving model to Weights-001--0.13649.hdf5
1/1 [==============================] - 2s 2s/step - loss: 0.8792 - mean_absolute_error: 0.8792 - val_loss: 0.1365 - val_mean_absolute_error: 0.1365
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8470 - mean_absolute_error: 0.8470
Epoch 2: val_loss did not improve from 0.13649
1/1 [==============================] - 0s 51ms/step - loss: 0.8470 - mean_absolute_error: 0.8470 - val_loss: 0.2232 - val_mean_absolute_error: 0.2232
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8231 - mean_absolute_error: 0.8231
Epoch 3: val_loss did not improve from 0.13649
1/1 [==============================] - 0s 57ms/step - loss: 0.8231 - mean_absolute_error: 0.8231 - val_loss: 0.3145 - val_mean_absolute_error: 0.3145
Epoch 4/500
1/1 [==============================] - ETA: 

Epoch 28/500
1/1 [==============================] - ETA: 0s - loss: 0.7787 - mean_absolute_error: 0.7787
Epoch 28: val_loss did not improve from 0.13649
1/1 [==============================] - 0s 74ms/step - loss: 0.7787 - mean_absolute_error: 0.7787 - val_loss: 0.3043 - val_mean_absolute_error: 0.3043
Epoch 29/500
1/1 [==============================] - ETA: 0s - loss: 0.7731 - mean_absolute_error: 0.7731
Epoch 29: val_loss did not improve from 0.13649
1/1 [==============================] - 0s 68ms/step - loss: 0.7731 - mean_absolute_error: 0.7731 - val_loss: 0.3244 - val_mean_absolute_error: 0.3244
Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 0.7483 - mean_absolute_error: 0.7483
Epoch 30: val_loss did not improve from 0.13649
1/1 [==============================] - 0s 65ms/step - loss: 0.7483 - mean_absolute_error: 0.7483 - val_loss: 0.4266 - val_mean_absolute_error: 0.4266
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 0.7202 - mean_absolute_

Epoch 55/500
1/1 [==============================] - ETA: 0s - loss: 0.7911 - mean_absolute_error: 0.7911
Epoch 55: val_loss did not improve from 0.12344
1/1 [==============================] - 0s 75ms/step - loss: 0.7911 - mean_absolute_error: 0.7911 - val_loss: 0.2188 - val_mean_absolute_error: 0.2188
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.7965 - mean_absolute_error: 0.7965
Epoch 56: val_loss did not improve from 0.12344
1/1 [==============================] - 0s 85ms/step - loss: 0.7965 - mean_absolute_error: 0.7965 - val_loss: 0.2001 - val_mean_absolute_error: 0.2001
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.8022 - mean_absolute_error: 0.8022
Epoch 57: val_loss did not improve from 0.12344
1/1 [==============================] - 0s 68ms/step - loss: 0.8022 - mean_absolute_error: 0.8022 - val_loss: 0.1878 - val_mean_absolute_error: 0.1878
Epoch 58/500
1/1 [==============================] - ETA: 0s - loss: 0.8074 - mean_absolute_

1/1 [==============================] - ETA: 0s - loss: 0.7837 - mean_absolute_error: 0.7837
Epoch 82: val_loss did not improve from 0.12344
1/1 [==============================] - 0s 60ms/step - loss: 0.7837 - mean_absolute_error: 0.7837 - val_loss: 0.3105 - val_mean_absolute_error: 0.3105
Epoch 83/500
1/1 [==============================] - ETA: 0s - loss: 0.7809 - mean_absolute_error: 0.7809
Epoch 83: val_loss did not improve from 0.12344
1/1 [==============================] - 0s 54ms/step - loss: 0.7809 - mean_absolute_error: 0.7809 - val_loss: 0.3107 - val_mean_absolute_error: 0.3107
Epoch 84/500
1/1 [==============================] - ETA: 0s - loss: 0.7777 - mean_absolute_error: 0.7777
Epoch 84: val_loss did not improve from 0.12344
1/1 [==============================] - 0s 54ms/step - loss: 0.7777 - mean_absolute_error: 0.7777 - val_loss: 0.3110 - val_mean_absolute_error: 0.3110
Epoch 85/500
1/1 [==============================] - ETA: 0s - loss: 0.7740 - mean_absolute_error: 0.7740

1/1 [==============================] - 0s 54ms/step - loss: 0.5067 - mean_absolute_error: 0.5067 - val_loss: 0.1725 - val_mean_absolute_error: 0.1725
Epoch 109/500
1/1 [==============================] - ETA: 0s - loss: 0.5446 - mean_absolute_error: 0.5446
Epoch 109: val_loss did not improve from 0.02715
1/1 [==============================] - 0s 50ms/step - loss: 0.5446 - mean_absolute_error: 0.5446 - val_loss: 0.4438 - val_mean_absolute_error: 0.4438
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.8045 - mean_absolute_error: 0.8045
Epoch 110: val_loss improved from 0.02715 to 0.02337, saving model to Weights-110--0.02337.hdf5
1/1 [==============================] - 0s 98ms/step - loss: 0.8045 - mean_absolute_error: 0.8045 - val_loss: 0.0234 - val_mean_absolute_error: 0.0234
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.4695 - mean_absolute_error: 0.4695
Epoch 111: val_loss did not improve from 0.02337
1/1 [==============================] -

...

Epoch 378/500
1/1 [==============================] - ETA: 0s - loss: 0.1444 - mean_absolute_error: 0.1444
Epoch 378: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.1444 - mean_absolute_error: 0.1444 - val_loss: 0.1802 - val_mean_absolute_error: 0.1802
Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.1844 - mean_absolute_error: 0.1844
Epoch 379: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 51ms/step - loss: 0.1844 - mean_absolute_error: 0.1844 - val_loss: 0.2058 - val_mean_absolute_error: 0.2058
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.2180 - mean_absolute_error: 0.2180
Epoch 380: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 55ms/step - loss: 0.2180 - mean_absolute_error: 0.2180 - val_loss: 0.0077 - val_mean_absolute_error: 0.0077
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.1152 - mean_ab

Epoch 405/500
1/1 [==============================] - ETA: 0s - loss: 0.1425 - mean_absolute_error: 0.1425
Epoch 405: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 55ms/step - loss: 0.1425 - mean_absolute_error: 0.1425 - val_loss: 0.1311 - val_mean_absolute_error: 0.1311
Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.1594 - mean_absolute_error: 0.1594
Epoch 406: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 51ms/step - loss: 0.1594 - mean_absolute_error: 0.1594 - val_loss: 0.1150 - val_mean_absolute_error: 0.1150
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.1176 - mean_absolute_error: 0.1176
Epoch 407: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 53ms/step - loss: 0.1176 - mean_absolute_error: 0.1176 - val_loss: 0.0185 - val_mean_absolute_error: 0.0185
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0638 - mean_ab

Epoch 432/500
1/1 [==============================] - ETA: 0s - loss: 0.0818 - mean_absolute_error: 0.0818
Epoch 432: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 55ms/step - loss: 0.0818 - mean_absolute_error: 0.0818 - val_loss: 0.0589 - val_mean_absolute_error: 0.0589
Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.1432 - mean_absolute_error: 0.1432
Epoch 433: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.1432 - mean_absolute_error: 0.1432 - val_loss: 0.0470 - val_mean_absolute_error: 0.0470
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.1302 - mean_absolute_error: 0.1302
Epoch 434: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 54ms/step - loss: 0.1302 - mean_absolute_error: 0.1302 - val_loss: 0.0753 - val_mean_absolute_error: 0.0753
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0937 - mean_ab

Epoch 459/500
1/1 [==============================] - ETA: 0s - loss: 0.0592 - mean_absolute_error: 0.0592
Epoch 459: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 50ms/step - loss: 0.0592 - mean_absolute_error: 0.0592 - val_loss: 0.0451 - val_mean_absolute_error: 0.0451
Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0504 - mean_absolute_error: 0.0504
Epoch 460: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 49ms/step - loss: 0.0504 - mean_absolute_error: 0.0504 - val_loss: 0.0414 - val_mean_absolute_error: 0.0414
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0500 - mean_absolute_error: 0.0500
Epoch 461: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 48ms/step - loss: 0.0500 - mean_absolute_error: 0.0500 - val_loss: 0.0249 - val_mean_absolute_error: 0.0249
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0729 - mean_ab

Epoch 486/500
1/1 [==============================] - ETA: 0s - loss: 0.0871 - mean_absolute_error: 0.0871
Epoch 486: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 63ms/step - loss: 0.0871 - mean_absolute_error: 0.0871 - val_loss: 0.0165 - val_mean_absolute_error: 0.0165
Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0844 - mean_absolute_error: 0.0844
Epoch 487: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 69ms/step - loss: 0.0844 - mean_absolute_error: 0.0844 - val_loss: 0.0756 - val_mean_absolute_error: 0.0756
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0934 - mean_absolute_error: 0.0934
Epoch 488: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 69ms/step - loss: 0.0934 - mean_absolute_error: 0.0934 - val_loss: 0.0851 - val_mean_absolute_error: 0.0851
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0729 - mean_ab

In [266]:
NN_model2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [267]:
y_pred2=NN_model2.predict(X_test)
r2_score(y_test,y_pred2)

1/1 [==============================] - 0s 57ms/step


0.9436923439509783

In [268]:
y_pred_train2=NN_model2.predict(X_train)
r2_score(y_train,y_pred_train2)

1/1 [==============================] - 0s 16ms/step


0.9914940862008916

In [278]:
from math import sqrt
print("Train exp Mean absolute error :", mean_absolute_error(y_train,y_pred_train2))
print("Train exp Mean squared error :",mean_squared_error(y_train,y_pred_train2))
print("Train exp Root Mean squared error :",sqrt(mean_squared_error(y_train,y_pred_train2)))

Train exp Mean absolute error : 0.07554005184656068
Train exp Mean squared error : 0.008505913799108419
Train exp Root Mean squared error : 0.09222751107510394


In [286]:
y_pred_real=NN_model2.predict(X1)

1/1 [==============================] - 0s 16ms/step


In [287]:
y_real1 = sc_y.inverse_transform(y1)
pred_real1= sc_y.inverse_transform(y_pred_real.reshape(-1,1))
ann_prediction_real1=get_preds(y_real1, pred_real1)
ann_prediction_real1

,Actual,Predicted
0,7.22,7.224477
1,7.52,7.479414
2,4.58,4.530869
3,6.45,6.356785
4,4.47,4.239712
5,5.67,5.865943
6,6.46,6.446592
7,7.76,7.623835
8,6.65,6.635098
9,4.52,4.431186


In [288]:
r2_score(y1,y_pred_real)

0.9802848438083835

In [272]:
NN_model3= Sequential()

In [273]:
# The Input Layer :
NN_model3.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='selu'))

# The Hidden Layers :
NN_model3.add(Dense(64, kernel_initializer='normal',activation='selu'))
NN_model3.add(Dense(32, kernel_initializer='normal',activation='selu'))
NN_model3.add(Dense(16, kernel_initializer='normal',activation='selu'))

# The Output Layer :
NN_model3.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model3.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model3.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_135 (Dense)           (None, 128)               512       
                                                                 
 dense_136 (Dense)           (None, 64)                8256      
                                                                 
 dense_137 (Dense)           (None, 32)                2080      
                                                                 
 dense_138 (Dense)           (None, 16)                528       
                                                                 
 dense_139 (Dense)           (None, 1)                 17        
                                                                 
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [274]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint3 = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list3 = [checkpoint3]

In [275]:
NN_model3.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.1, callbacks=callbacks_list2)

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 0.8331 - mean_absolute_error: 0.8331
Epoch 1: val_loss did not improve from 0.00482
1/1 [==============================] - 1s 589ms/step - loss: 0.8331 - mean_absolute_error: 0.8331 - val_loss: 0.1858 - val_mean_absolute_error: 0.1858
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 0.8290 - mean_absolute_error: 0.8290
Epoch 2: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 26ms/step - loss: 0.8290 - mean_absolute_error: 0.8290 - val_loss: 0.1888 - val_mean_absolute_error: 0.1888
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 0.8247 - mean_absolute_error: 0.8247
Epoch 3: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 24ms/step - loss: 0.8247 - mean_absolute_error: 0.8247 - val_loss: 0.1921 - val_mean_absolute_error: 0.1921
Epoch 4/500
1/1 [==============================] - ETA: 0s - loss: 0.8201 - mean_absolute_error:

1/1 [==============================] - ETA: 0s - loss: 0.1390 - mean_absolute_error: 0.1390
Epoch 55: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 24ms/step - loss: 0.1390 - mean_absolute_error: 0.1390 - val_loss: 0.1184 - val_mean_absolute_error: 0.1184
Epoch 56/500
1/1 [==============================] - ETA: 0s - loss: 0.1368 - mean_absolute_error: 0.1368
Epoch 56: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.1368 - mean_absolute_error: 0.1368 - val_loss: 0.1172 - val_mean_absolute_error: 0.1172
Epoch 57/500
1/1 [==============================] - ETA: 0s - loss: 0.1328 - mean_absolute_error: 0.1328
Epoch 57: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 25ms/step - loss: 0.1328 - mean_absolute_error: 0.1328 - val_loss: 0.1194 - val_mean_absolute_error: 0.1194
Epoch 58/500
1/1 [==============================] - ETA: 0s - loss: 0.1270 - mean_absolute_error: 0.1270

1/1 [==============================] - ETA: 0s - loss: 0.0703 - mean_absolute_error: 0.0703
Epoch 109: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 29ms/step - loss: 0.0703 - mean_absolute_error: 0.0703 - val_loss: 0.0293 - val_mean_absolute_error: 0.0293
Epoch 110/500
1/1 [==============================] - ETA: 0s - loss: 0.0660 - mean_absolute_error: 0.0660
Epoch 110: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 31ms/step - loss: 0.0660 - mean_absolute_error: 0.0660 - val_loss: 0.0388 - val_mean_absolute_error: 0.0388
Epoch 111/500
1/1 [==============================] - ETA: 0s - loss: 0.0595 - mean_absolute_error: 0.0595
Epoch 111: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 28ms/step - loss: 0.0595 - mean_absolute_error: 0.0595 - val_loss: 0.0505 - val_mean_absolute_error: 0.0505
Epoch 112/500
1/1 [==============================] - ETA: 0s - loss: 0.0551 - mean_absolute_error: 

Epoch 136/500
1/1 [==============================] - ETA: 0s - loss: 0.0461 - mean_absolute_error: 0.0461
Epoch 136: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0461 - mean_absolute_error: 0.0461 - val_loss: 0.0570 - val_mean_absolute_error: 0.0570
Epoch 137/500
1/1 [==============================] - ETA: 0s - loss: 0.0406 - mean_absolute_error: 0.0406
Epoch 137: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 24ms/step - loss: 0.0406 - mean_absolute_error: 0.0406 - val_loss: 0.0612 - val_mean_absolute_error: 0.0612
Epoch 138/500
1/1 [==============================] - ETA: 0s - loss: 0.0398 - mean_absolute_error: 0.0398
Epoch 138: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 22ms/step - loss: 0.0398 - mean_absolute_error: 0.0398 - val_loss: 0.0636 - val_mean_absolute_error: 0.0636
Epoch 139/500
1/1 [==============================] - ETA: 0s - loss: 0.0398 - mean_ab

Epoch 163/500
1/1 [==============================] - ETA: 0s - loss: 0.0411 - mean_absolute_error: 0.0411
Epoch 163: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0411 - mean_absolute_error: 0.0411 - val_loss: 0.0555 - val_mean_absolute_error: 0.0555
Epoch 164/500
1/1 [==============================] - ETA: 0s - loss: 0.0443 - mean_absolute_error: 0.0443
Epoch 164: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 22ms/step - loss: 0.0443 - mean_absolute_error: 0.0443 - val_loss: 0.0621 - val_mean_absolute_error: 0.0621
Epoch 165/500
1/1 [==============================] - ETA: 0s - loss: 0.0315 - mean_absolute_error: 0.0315
Epoch 165: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0315 - mean_absolute_error: 0.0315 - val_loss: 0.0750 - val_mean_absolute_error: 0.0750
Epoch 166/500
1/1 [==============================] - ETA: 0s - loss: 0.0379 - mean_ab

...

Epoch 379/500
1/1 [==============================] - ETA: 0s - loss: 0.0107 - mean_absolute_error: 0.0107
Epoch 379: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 24ms/step - loss: 0.0107 - mean_absolute_error: 0.0107 - val_loss: 0.0570 - val_mean_absolute_error: 0.0570
Epoch 380/500
1/1 [==============================] - ETA: 0s - loss: 0.0102 - mean_absolute_error: 0.0102
Epoch 380: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0102 - mean_absolute_error: 0.0102 - val_loss: 0.0616 - val_mean_absolute_error: 0.0616
Epoch 381/500
1/1 [==============================] - ETA: 0s - loss: 0.0059 - mean_absolute_error: 0.0059
Epoch 381: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0059 - mean_absolute_error: 0.0059 - val_loss: 0.0697 - val_mean_absolute_error: 0.0697
Epoch 382/500
1/1 [==============================] - ETA: 0s - loss: 0.0126 - mean_ab

Epoch 406/500
1/1 [==============================] - ETA: 0s - loss: 0.0083 - mean_absolute_error: 0.0083
Epoch 406: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0083 - mean_absolute_error: 0.0083 - val_loss: 0.0725 - val_mean_absolute_error: 0.0725
Epoch 407/500
1/1 [==============================] - ETA: 0s - loss: 0.0089 - mean_absolute_error: 0.0089
Epoch 407: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 25ms/step - loss: 0.0089 - mean_absolute_error: 0.0089 - val_loss: 0.0627 - val_mean_absolute_error: 0.0627
Epoch 408/500
1/1 [==============================] - ETA: 0s - loss: 0.0064 - mean_absolute_error: 0.0064
Epoch 408: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 22ms/step - loss: 0.0064 - mean_absolute_error: 0.0064 - val_loss: 0.0519 - val_mean_absolute_error: 0.0519
Epoch 409/500
1/1 [==============================] - ETA: 0s - loss: 0.0091 - mean_ab

Epoch 433/500
1/1 [==============================] - ETA: 0s - loss: 0.0071 - mean_absolute_error: 0.0071
Epoch 433: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0071 - mean_absolute_error: 0.0071 - val_loss: 0.0702 - val_mean_absolute_error: 0.0702
Epoch 434/500
1/1 [==============================] - ETA: 0s - loss: 0.0084 - mean_absolute_error: 0.0084
Epoch 434: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 22ms/step - loss: 0.0084 - mean_absolute_error: 0.0084 - val_loss: 0.0718 - val_mean_absolute_error: 0.0718
Epoch 435/500
1/1 [==============================] - ETA: 0s - loss: 0.0110 - mean_absolute_error: 0.0110
Epoch 435: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0110 - mean_absolute_error: 0.0110 - val_loss: 0.0641 - val_mean_absolute_error: 0.0641
Epoch 436/500
1/1 [==============================] - ETA: 0s - loss: 0.0042 - mean_ab

Epoch 460/500
1/1 [==============================] - ETA: 0s - loss: 0.0094 - mean_absolute_error: 0.0094
Epoch 460: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0094 - mean_absolute_error: 0.0094 - val_loss: 0.0586 - val_mean_absolute_error: 0.0586
Epoch 461/500
1/1 [==============================] - ETA: 0s - loss: 0.0048 - mean_absolute_error: 0.0048
Epoch 461: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0048 - mean_absolute_error: 0.0048 - val_loss: 0.0641 - val_mean_absolute_error: 0.0641
Epoch 462/500
1/1 [==============================] - ETA: 0s - loss: 0.0138 - mean_absolute_error: 0.0138
Epoch 462: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0138 - mean_absolute_error: 0.0138 - val_loss: 0.0631 - val_mean_absolute_error: 0.0631
Epoch 463/500
1/1 [==============================] - ETA: 0s - loss: 0.0174 - mean_ab

Epoch 487/500
1/1 [==============================] - ETA: 0s - loss: 0.0036 - mean_absolute_error: 0.0036
Epoch 487: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 23ms/step - loss: 0.0036 - mean_absolute_error: 0.0036 - val_loss: 0.0665 - val_mean_absolute_error: 0.0665
Epoch 488/500
1/1 [==============================] - ETA: 0s - loss: 0.0103 - mean_absolute_error: 0.0103
Epoch 488: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0103 - mean_absolute_error: 0.0103 - val_loss: 0.0662 - val_mean_absolute_error: 0.0662
Epoch 489/500
1/1 [==============================] - ETA: 0s - loss: 0.0069 - mean_absolute_error: 0.0069
Epoch 489: val_loss did not improve from 0.00482
1/1 [==============================] - 0s 21ms/step - loss: 0.0069 - mean_absolute_error: 0.0069 - val_loss: 0.0624 - val_mean_absolute_error: 0.0624
Epoch 490/500
1/1 [==============================] - ETA: 0s - loss: 0.0102 - mean_ab

In [276]:
NN_model3.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [277]:
y_pred3=NN_model3.predict(X_test)
y_pred_train3=NN_model3.predict(X_train)
r2_score(y_train,y_pred_train3) , r2_score(y_test,y_pred3)

1/1 [==============================] - 0s 14ms/step


(0.9991708791000046, 0.8414636580236003)

In [279]:
print("Train selu Mean absolute error :", mean_absolute_error(y_train,y_pred_train3))
print("Train selu Mean squared error :",mean_squared_error(y_train,y_pred_train3))
print("Train selu Root Mean squared error :",sqrt(mean_squared_error(y_train,y_pred_train3)))

Train selu Mean absolute error : 0.019257721660240354
Train selu Mean squared error : 0.0008291208999953842
Train selu Root Mean squared error : 0.028794459536434856


In [296]:
y_pred_real3=NN_model3.predict(X1)
r2_score(y1,y_pred_real3)

1/1 [==============================] - 0s 23ms/step


0.9611596896826472